In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

In [2]:
data_train = pd.read_csv('data_train.csv.zip', index_col=0, dtype=int)

In [3]:
# Последние 4 недели - validate data set; все, что ранее - train
val_sec = 28 * 24 * 3600  # 4 weeks
last_learn_sec = data_train['buy_time'].max() - val_sec
data_learn = data_train.query("buy_time <= @last_learn_sec")
data_validate = data_train.query("buy_time > @last_learn_sec")

In [4]:
# Вероятности предложения услуг при отказе
t0_vas_proba = data_learn.query("target==0")['vas_id'].value_counts(normalize=True, sort=False).sort_index()
# Вероятности предложения услуг при покупке
t1_vas_proba = data_learn.query("target==1")['vas_id'].value_counts(normalize=True, sort=False)
# Решающая таблица
vas_proba = pd.DataFrame({'vas_id': t0_vas_proba.index, 't0': t0_vas_proba, 't1': t1_vas_proba})
vas_proba['prediction'] = np.where(vas_proba['t1'] > vas_proba['t0'], 1, 0)
vas_proba

,vas_id,t0,t1,prediction
1,1,0.487827,0.141515,0
2,2,0.264070,0.110441,0
4,4,0.108136,0.501557,1
5,5,0.075741,0.039912,0
6,6,0.015310,0.167853,1
7,7,0.021351,0.005268,0
8,8,0.021612,0.008788,0
9,9,0.005953,0.024666,1


In [5]:
# Пробуем предсказать на основе решающей таблицы
data_validate_pred = data_validate.merge(vas_proba, on='vas_id')

In [6]:
f1_score(data_validate_pred.target, data_validate_pred.prediction, average='macro')

0.7465185858522339